In [13]:
import pyautogui as g
import time

cartas = [['2p','3p','4p','5p','6p','7p','8p','9p','dp','jp','qp','kp','ap'],
          ['2e','3e','4e','5e','6e','7e','8e','9e','de','je','qe','ke','ae'],
          ['2c','3c','4c','5c','6c','7c','8c','9c','dc','jc','qc','kc','ac'],
          ['2o','3o','4o','5o','6o','7o','8o','9o','do','jo','qo','ko','ao']]
mesa = ['kp','ko','6o']
mao = ['3o','2o']
# jogada na ordem
sequencia = []
# maiores sequencias de cartas
seq = [[],[],[]]
# straigh flush
strflush = []
# cartas para realizar jogada
jogada = [mao[0], mao[1], mesa[0], mesa[1], mesa[2]]
# numero de cartas para cada naipe : p;e;c;o
naipe = [0,0,0,0]
# maior naipe e o numero de cartas desse naipe
mnaipe = ['p',0]
pontos = 0
num = len(jogada)
# por as cartas em ordem crescente e contar o número de cartas de cada naipe
for i in jogada:
    # ver qual o naipe mais relevante
    if i[1] == 'p':
        naipe[0] = naipe[0] + 1
    if i[1] == 'e':
        naipe[1] = naipe[1] + 1
    if i[1] == 'c':
        naipe[2] = naipe[2] + 1
    if i[1] == 'o':
        naipe[3] = naipe[3] + 1
    sequencia.append(i)    
    # inserir as cartas na sequencia e as ordenar
    for n in range(len(sequencia) - 1):
        # verificar se a carta é um digito
        if i[0].isdigit():
            # verificar se a próxima carta na ordem é um número
            if sequencia[- n - 2][0].isdigit():
                # verificar se é o próximo número
                if sequencia[-n -1][0] < sequencia[-n -2][0]:
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            # já que o próximo não é um numero, ele vai na frente na ordem
            else:
                sequencia[- n - 1] = sequencia[- n - 2]
                sequencia[- n - 2] = i
        # verificar se o próximo digito também não é um número
        elif sequencia[- n - 2][0].isdigit() == False:
            # se o digito for d, o próximo deverá ser j, k, q ou a
            if sequencia[- n - 1][0] == 'd':
                if sequencia[- n - 2][0] == 'j' or sequencia[- n - 2][0] == 'q' or sequencia[- n - 2][0] == 'k' or sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            if sequencia[- n - 1][0] == 'j':
                if sequencia[- n - 2][0] == 'q' or sequencia[- n - 2][0] == 'k' or sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            if sequencia[- n - 1][0] == 'q':
                if sequencia[- n - 2][0] == 'k' or sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            if sequencia[- n - 1][0] == 'k':
                if sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i

# determinar qual é o naipe mais importante e quantas cartas desse naipe tem
mnaipe[0] = 'p'
mnaipe[1] = naipe[0]
if mnaipe[1] < naipe[1]:
    mnaipe[0] = 'e'
    mnaipe[1] = naipe[1]
if mnaipe[1] < naipe[2]:
    mnaipe[0] = 'c'
    mnaipe[1] = naipe[2]
if mnaipe[1] < naipe[3]:
    mnaipe[0] = 'o'
    mnaipe[1] = naipe[3]
        

# conferir straight
seq[0].append(sequencia[0])

if sequencia[1][1] != mnaipe[0] and sequencia[0][0] == sequencia[1][0] and sequencia[0][1] == mnaipe[0]:
    seq[1].append(sequencia[0])
else:
    seq[1].append(sequencia[1])
    
if sequencia[2][1] != mnaipe[0]:
    if sequencia[0][0] == sequencia[2][0] and sequencia[0][1] == mnaipe[0]:
        seq[2].append(sequencia[0])
    elif sequencia[1][0] == sequencia[2][0] and sequencia[1][1] == mnaipe[0]:
        seq[2].append(sequencia[1])
    else:
        seq[2].append(sequencia[2])
else:
    seq[2].append(sequencia[2])

# cartas por naipe das sequencias
naip = [[0,0,0,0],[0,0,0,0],[0,0,0,0]]
for i in range(3):
    # conferir cada dígito para checar se o próximo está na ordem
    for n in range(len(jogada) - 1 - i):
        # ver se o dígito é menor que 9, pois se não for, não poderá ser comparado com outro dígito
        if sequencia[n + i][0].isdigit() and float(sequencia[n + i][0]) < 9:
            # verificar se o próximo é um número
            if sequencia[n + 1 + i][0].isdigit():
                # verificar se é o próximo número da ordem
                if float(sequencia[n + i][0]) + 1 == float(sequencia[n + 1 + i][0]):
                    seq[i].append(sequencia[n + 1 + i])
                # verificar se é um número igual
                elif sequencia[n + i][0] == sequencia[n + 1 + i][0]:
                    # verificar se essa carta tem um naipe mais importante
                    if sequencia[n + 1 + i][1] == mnaipe[0]:
                        seq[i].pop()
                        seq[i].append(sequencia[n + 1 + i])
                elif float(sequencia[n + i][0]) != float(sequencia[n + 1 + i][0]):
                    break
            else:
                break
        # se o dígito for 9, o próximo deverá ser o d
        elif sequencia[n + i][0] == '9' and sequencia[n + 1 + i][0] == 'd':
            seq[i].append(sequencia[n + 1 + i])    
        elif sequencia[n + i][0] == 'd' and sequencia[n + 1 + i][0] == 'j':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == 'j' and sequencia[n + 1 + i][0] == 'q':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == 'q' and sequencia[n + 1 + i][0] == 'k':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == 'k' and sequencia[n + 1 + i][0] == 'a':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == sequencia[n + 1 + i][0]:
            if sequencia[n + 1 + i][1] == mnaipe[0]:
                seq[i].pop()
                seq[i].append(sequencia[n + 1 + i])
        else:
            break
            
    # conferir straight flush
    if len(seq[i]) > 2:
        # conferir o número de cartas por naipe das sequencias
        naip = [[0,0,0,0],[0,0,0,0],[0,0,0,0]]
        for n in seq[i]:
            if n[1] == 'p':
                naip[i][0] = naip[i][0] + 1
            if n[1] == 'e':
                naip[i][1] = naip[i][1] + 1
            if n[1] == 'c':
                naip[i][2] = naip[i][2] + 1
            if n[1] == 'o':
                naip[i][3] = naip[i][3] + 1
        # caso haja flush
        if len(seq[i]) > 4:
            if naip[i][0] > 4 or naip[i][1] > 4 or naip[i][2] > 4 or naip[i][3] > 4:
                for n in seq[i]:
                    if n[1] != mnaipe[0]:
                        seq[i].pop(seq[i].index(n))
                # caso todas, ou quase todas, as cartas sejam do mesmo naipe
                if len(seq[i]) == 7:
                    seq[i].pop(0)
                    seq[i].pop(0)
                elif len(seq[i]) == 6:
                    seq[i].pop(0)
                strflush = seq[i]
                
# conferir quadra, trinca, duplas e dupla
reps = sequencia
quadra = []
trinca = []
pares = []
par = []
for i in range(len(jogada)):
    if reps == []:
        break
    # copiar o valor da carta a ser testada e apagar a original do array
    test = reps[-1]
    reps.pop(-1)
    if reps == []:
        break
    quadra.append(reps[-1])
    trinca.append(reps[-1])
    pares.append(reps[-1])
    par.append(reps[-1])
    for n in range(len(jogada) - 1):
        # comparar todas as cartas à testada
        if reps == []:
            break
        if reps[-1][0] == test[0]:
            quadra.append(reps[-1])
            trinca.append(reps[-1])
            # verificar se está se criando outro par utilizando uma carta que já está no primeiro par
            if len(pares) > 1:
                if pares[1][0] != reps[-1][0]:
                    pares.append(reps[-1])
            else:
                pares.append(reps[-1])
            par.append(reps[-1])
            reps.pop(-1)
    # se a carta testada não preencheu uma jogada, o processo recomeça com outra carta
    if len(quadra) < 4:
        quadra = []
    if len(trinca) < 3:
        trinca = []
    if len(pares) < 2:
        pares = []
    if len(pares) == 3:
        pares.pop(2)
    if len(par) < 2:
        par = []

if strflush != []:
    print('straight flush com maior carta :', strflush[4][0], 'de naipe :', strflush[4][1])
    pontos = 8
    
elif len(quadra) > 3:
    print('quadra de :', quadra[3][0])
    pontos = 7
    
elif len(trinca) > 2 and len(pares) > 3:
    if pares[3][0] == trinca[2][0]:
        print('full house com uma trinca de :', trinca[2][0], 'e uma dupla de :', pares[1][0])
        pontos = 6
    else:
        print('full house com uma trinca de :', trinca[2][0], 'e uma dupla de :', pares[3][0])
        pontos = 6
        
elif mnaipe[1] > 4:
    print('flush do naipe :', mnaipe[0])
    pontos = 5

elif len(seq[0]) > 4 or len(seq[1]) > 4 or len(seq[2]) > 4:
    for i in range(3):
        if len(seq[-i - 1]) > 4:
            print('sequência com a maior carta sendo :', seq[-i -1][-1][0])
            pontos = 4
            break
        
elif len(trinca) > 2:
    print('trinca de :', trinca[2][0])
    pontos = 3
    
elif len(pares) > 3:
    print('1 par de :', pares[1][0], 'e outro de :', pares[3][0])
    pontos = 2
    
elif len(par) > 1:
    print('par de :', par[1][0])
    pontos = 1
    
cpar = 0
cpares = 0
ctrinca = 0
cseq = 0
cflush = 0
cfull = 0
cquadra = 0
cstrflush = 0
        
if pontos == 0 and num != 7:
    if num == 5:
        cpar = 1056/2162
        cpares = 180/2162
        ctrinca = 30/2162
    elif num == 6:
        cpar = 18/46

elif pontos == 1:
    if num == 5:
        cpares = 810/2162
        ctrinca = 144/2162
        cfull = 54/2162
        cquadra = 2/2162
    elif num == 6:
        cpares = 12/46
        ctrinca = 2/46

elif pontos == 2:
    if num == 5:
        cfull = 358/2162
        cquadra = 4/2162
    elif num == 6:
        if len(pares) > 5:
            cfull = 6/46
        else:
            cfull = 4/46

elif pontos == 3:
    if num == 5:
        cfull = 630/2162
        cquadra = 92/2162
    elif num == 6:
        cquadra = 1/46
        cfull = 9/46

elif pontos == 6:
    if num == 5:
        cquadra = 94/2162
    elif num == 6:
        if len(trinca) > 5:
            cquadra = 2/46
        else:
            cquadra = 1/46

if pontos < 5:
    if num == 5:
        if mnaipe[1] == 3:
            cflush = (10*9)/(47*46)
        elif mnaipe[1] == 4:
            cflush = 1 - (38*37)/(47*46)
    elif num == 6 and mnaipe[1] == 4:
        cflush = 9/46
        
# chance de conseguir sequencia        
highmatch1 = [0,0]
if pontos < 4:
    sequencias1 = [['2','3','4','5','6'],['3','4','5','6','7'],['4','5','6','7','8'],
                  ['5','6','7','8','9'],['6','7','8','9','d'],['7','8','9','d','j'],
                  ['8','9','d','j','q'],['9','d','j','q','k'],['d','j','q','k','a']]
    for i in sequencias1:
        match1 = 0
        for n in i:
            rep = ['0', '0']
            repnum = 0
            for c in jogada:
                if c[0] == n and c[0] != rep[0] and c[0] != rep[1]:
                    match1 = match1 + 1
                    if repnum == 0:
                        repnum = 1
                        rep[0] = c[0]
                    else:
                        repnum = 0
                        rep[1] = c[0]
        if match1 >= highmatch1[0]:
            highmatch1[0] = match1
            highmatch1[1] = sequencias1.index(i)
    if num == 5:
        if highmatch1[0] == 4:
            if highmatch1[1] == 0 or highmatch1[1] == 8:
                cseq = 1 - (43*42)/(47*46)
            else:
                cseq = 1 - (39*38)/(47*46)
        elif highmatch1 == 3:
            if highmatch1[1] == 0 or highmatch1[1] == 8:
                cseq = (8*4)/(47*46)
            elif highmatch1[1] == 1 or highmatch1[1] == 7:
                cseq = 2*(8*4)/(47*46)
            else:
                cseq = 3*(8*4)/(47*46)
    if num == 6 and highmatch1[0] == 4:
        if highmatch1[1] == 0 or highmatch1[1] == 8:
            cseq = 4/46
        else:
            cseq = 8/46

# chance de conseguir straight flush
highmatch2 = [0,0]
if pontos < 8:
    sequencias2 = [['2','3','4','5','6'],['3','4','5','6','7'],['4','5','6','7','8'],
                  ['5','6','7','8','9'],['6','7','8','9','d'],['7','8','9','d','j'],
                  ['8','9','d','j','q'],['9','d','j','q','k'],['d','j','q','k','a']]
    for i in sequencias2:
        match2 = 0
        for n in i:
            for c in jogada:
                if c[0] == n and c[1] == mnaipe[0]:
                    match2 = match2 + 1
                        
        if match2 >= highmatch2[0]:
            highmatch2[0] = match2
            highmatch2[1] = sequencias2.index(i)
    if num == 5:
        if highmatch2[0] == 4:
            if highmatch2[1] == 0 or highmatch2[1] == 8:
                cstrflush = 1 - (46*45)/(47*46)
            else:
                cstrflush = 1 - (45*44)/(47*46)
        elif highmatch2 == 3:
            if highmatch2[1] == 0 or highmatch2[1] == 8:
                cstrflush = (2*1)/(47*46)
            elif highmatch2[1] == 1 or highmatch2[1] == 7:
                cstrflush = 2*(2*1)/(47*46)
            else:
                cstrflush = 3*(2*1)/(47*46)
    if num == 6 and highmatch2[0] == 4:
        if highmatch2[1] == 0 or highmatch2[1] == 8:
            cstrflush = 4/46
        else:
            cstrflush = 8/46        
            
if pontos < 1:     
    print('Par :', round(cpar*100, 2), '%')
if pontos < 2:
    print('Pares :', round(cpares*100, 2), '%')
if pontos < 3:
    print('Trinca :', round(ctrinca*100,2), '%')
if pontos < 4:
    print('Sequência :', round(cseq*100, 2), '%')
if pontos < 5:
    print('Flush :', round(cflush*100, 2), '%')
if pontos < 6:
    print('Full house :', round(cfull*100, 2), '%')
if pontos < 7:
    print('Quadra :', round(cquadra*100, 2), '%')
if pontos < 8:
    print('Straight flush :', round(cstrflush*100, 2), '%')

    
    
    
    
print('//') 
print('//')
    
    
    

# testar cartas da mesa
sequencia = []
# maiores sequencias de cartas
seq = [[],[],[]]
# straigh flush
strflush = []
# cartas para realizar jogada
# numero de cartas para cada naipe : p;e;c;o
naipe = [0,0,0,0]
# maior naipe e o numero de cartas desse naipe
mnaipe = ['p',0]
pontos = 0
num = len(mesa)
# por as cartas em ordem crescente e contar o número de cartas de cada naipe
for i in mesa:
    # ver qual o naipe mais relevante
    if i[1] == 'p':
        naipe[0] = naipe[0] + 1
    if i[1] == 'e':
        naipe[1] = naipe[1] + 1
    if i[1] == 'c':
        naipe[2] = naipe[2] + 1
    if i[1] == 'o':
        naipe[3] = naipe[3] + 1
    sequencia.append(i)    
    # inserir as cartas na sequencia e as ordenar
    for n in range(len(sequencia) - 1):
        # verificar se a carta é um digito
        if i[0].isdigit():
            # verificar se a próxima carta na ordem é um número
            if sequencia[- n - 2][0].isdigit():
                # verificar se é o próximo número
                if sequencia[-n -1][0] < sequencia[-n -2][0]:
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            # já que o próximo não é um numero, ele vai na frente na ordem
            else:
                sequencia[- n - 1] = sequencia[- n - 2]
                sequencia[- n - 2] = i
        # verificar se o próximo digito também não é um número
        elif sequencia[- n - 2][0].isdigit() == False:
            # se o digito for d, o próximo deverá ser j, k, q ou a
            if sequencia[- n - 1][0] == 'd':
                if sequencia[- n - 2][0] == 'j' or sequencia[- n - 2][0] == 'q' or sequencia[- n - 2][0] == 'k' or sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            if sequencia[- n - 1][0] == 'j':
                if sequencia[- n - 2][0] == 'q' or sequencia[- n - 2][0] == 'k' or sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            if sequencia[- n - 1][0] == 'q':
                if sequencia[- n - 2][0] == 'k' or sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i
            if sequencia[- n - 1][0] == 'k':
                if sequencia[- n - 2][0] == 'a':
                    sequencia[- n - 1] = sequencia[- n - 2]
                    sequencia[- n - 2] = i

# determinar qual é o naipe mais importante e quantas cartas desse naipe tem
mnaipe[0] = 'p'
mnaipe[1] = naipe[0]
if mnaipe[1] < naipe[1]:
    mnaipe[0] = 'e'
    mnaipe[1] = naipe[1]
if mnaipe[1] < naipe[2]:
    mnaipe[0] = 'c'
    mnaipe[1] = naipe[2]
if mnaipe[1] < naipe[3]:
    mnaipe[0] = 'o'
    mnaipe[1] = naipe[3]
        

# conferir straight
seq[0].append(sequencia[0])

if sequencia[1][1] != mnaipe[0] and sequencia[0][0] == sequencia[1][0] and sequencia[0][1] == mnaipe[0]:
    seq[1].append(sequencia[0])
else:
    seq[1].append(sequencia[1])
    
if sequencia[2][1] != mnaipe[0]:
    if sequencia[0][0] == sequencia[2][0] and sequencia[0][1] == mnaipe[0]:
        seq[2].append(sequencia[0])
    elif sequencia[1][0] == sequencia[2][0] and sequencia[1][1] == mnaipe[0]:
        seq[2].append(sequencia[1])
    else:
        seq[2].append(sequencia[2])
else:
    seq[2].append(sequencia[2])

# cartas por naipe das sequencias
naip = [[0,0,0,0],[0,0,0,0],[0,0,0,0]]
for i in range(3):
    # conferir cada dígito para checar se o próximo está na ordem
    for n in range(len(mesa) - 1 - i):
        # ver se o dígito é menor que 9, pois se não for, não poderá ser comparado com outro dígito
        if sequencia[n + i][0].isdigit() and float(sequencia[n + i][0]) < 9:
            # verificar se o próximo é um número
            if sequencia[n + 1 + i][0].isdigit():
                # verificar se é o próximo número da ordem
                if float(sequencia[n + i][0]) + 1 == float(sequencia[n + 1 + i][0]):
                    seq[i].append(sequencia[n + 1 + i])
                # verificar se é um número igual
                elif sequencia[n + i][0] == sequencia[n + 1 + i][0]:
                    # verificar se essa carta tem um naipe mais importante
                    if sequencia[n + 1 + i][1] == mnaipe[0]:
                        seq[i].pop()
                        seq[i].append(sequencia[n + 1 + i])
                elif float(sequencia[n + i][0]) != float(sequencia[n + 1 + i][0]):
                    break
            else:
                break
        # se o dígito for 9, o próximo deverá ser o d
        elif sequencia[n + i][0] == '9' and sequencia[n + 1 + i][0] == 'd':
            seq[i].append(sequencia[n + 1 + i])    
        elif sequencia[n + i][0] == 'd' and sequencia[n + 1 + i][0] == 'j':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == 'j' and sequencia[n + 1 + i][0] == 'q':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == 'q' and sequencia[n + 1 + i][0] == 'k':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == 'k' and sequencia[n + 1 + i][0] == 'a':
            seq[i].append(sequencia[n + 1 + i])
        elif sequencia[n + i][0] == sequencia[n + 1 + i][0]:
            if sequencia[n + 1 + i][1] == mnaipe[0]:
                seq[i].pop()
                seq[i].append(sequencia[n + 1 + i])
        else:
            break
            
    # conferir straight flush
    if len(seq[i]) > 2:
        # conferir o número de cartas por naipe das sequencias
        naip = [[0,0,0,0],[0,0,0,0],[0,0,0,0]]
        for n in seq[i]:
            if n[1] == 'p':
                naip[i][0] = naip[i][0] + 1
            if n[1] == 'e':
                naip[i][1] = naip[i][1] + 1
            if n[1] == 'c':
                naip[i][2] = naip[i][2] + 1
            if n[1] == 'o':
                naip[i][3] = naip[i][3] + 1
        # caso haja flush
        if len(seq[i]) > 4:
            if naip[i][0] > 4 or naip[i][1] > 4 or naip[i][2] > 4 or naip[i][3] > 4:
                for n in seq[i]:
                    if n[1] != mnaipe[0]:
                        seq[i].pop(seq[i].index(n))
                # caso todas, ou quase todas, as cartas sejam do mesmo naipe
                if len(seq[i]) == 7:
                    seq[i].pop(0)
                    seq[i].pop(0)
                elif len(seq[i]) == 6:
                    seq[i].pop(0)
                strflush = seq[i]
                
# conferir quadra, trinca, duplas e dupla
reps = sequencia
quadra = []
trinca = []
pares = []
par = []
for i in range(len(mesa)):
    if reps == []:
        break
    # copiar o valor da carta a ser testada e apagar a original do array
    test = reps[-1]
    reps.pop(-1)
    if reps == []:
        break
    quadra.append(reps[-1])
    trinca.append(reps[-1])
    pares.append(reps[-1])
    par.append(reps[-1])
    for n in range(len(mesa) - 1):
        # comparar todas as cartas à testada
        if reps == []:
            break
        if reps[-1][0] == test[0]:
            quadra.append(reps[-1])
            trinca.append(reps[-1])
            # verificar se está se criando outro par utilizando uma carta que já está no primeiro par
            if len(pares) > 1:
                if pares[1][0] != reps[-1][0]:
                    pares.append(reps[-1])
            else:
                pares.append(reps[-1])
            par.append(reps[-1])
            reps.pop(-1)
    # se a carta testada não preencheu uma jogada, o processo recomeça com outra carta
    if len(quadra) < 4:
        quadra = []
    if len(trinca) < 3:
        trinca = []
    if len(pares) < 2:
        pares = []
    if len(pares) == 3:
        pares.pop(2)
    if len(par) < 2:
        par = []

if strflush != []:
    print('todos têm um straight flush com maior carta :', strflush[4][0], 'de naipe :', strflush[4][1])
    pontos = 8
    
elif len(quadra) > 3:
    print('todos têm uma quadra de :', quadra[3][0])
    pontos = 7
    
elif len(trinca) > 2 and len(pares) > 3:
    if pares[3][0] == trinca[2][0]:
        print('todos têm um full house com uma trinca de :', trinca[2][0], 'e uma dupla de :', pares[1][0])
        pontos = 6
    else:
        print('todos têm um full house com uma trinca de :', trinca[2][0], 'e uma dupla de :', pares[3][0])
        pontos = 6
        
elif mnaipe[1] > 4:
    print('todos têm um flush do naipe :', mnaipe[0])
    pontos = 5

elif len(seq[0]) > 4 or len(seq[1]) > 4 or len(seq[2]) > 4:
    for i in range(3):
        if len(seq[-i - 1]) > 4:
            print('todos têm uma sequência com a maior carta sendo :', seq[-i -1][-1][0])
            pontos = 4
            break
        
elif len(trinca) > 2:
    print('todos têm uma trinca de :', trinca[2][0])
    pontos = 3
    
elif len(pares) > 3:
    print('todos têm um 1 par de :', pares[1][0], 'e outro de :', pares[3][0])
    pontos = 2
    
elif len(par) > 1:
    print('todos têm um par de :', par[1][0])
    pontos = 1
        
cpar = 0
cpares = 0
ctrinca = 0
cseq = 0
cflush = 0
cfull = 0
cquadra = 0
cstrflush = 0
        
if pontos == 0:
    if num == 3:
        cpar = 2488320/5085024
        cpares = 1036800/5085024
        ctrinca = 190080/5085024 
        cfull = 76032/5085024
        cquadra = 3552/5085024
    elif num == 4:
        cpar = 51840/103776
        cpares = 16200/103776
        ctrinca = 2808/103776
        cfull = 648/103776
        cquadra = 24/103776
    elif num == 5:
        cpar = 1056/2162
        cpares = 180/2162
        ctrinca = 30/2162

elif pontos == 1:
    if num == 3:
        cpares = 2138400/5085024
        ctrinca = 506880/5085024 
        cfull = 384912/5085024
        cquadra = 27312/5085024
    elif num == 4:
        cpares = 43200/103776
        ctrinca = 8640/103776 
        cfull = 5568/103776
        cquadra = 288/103776
    elif num == 5:
        cpares = 810/2162
        ctrinca = 144/2162
        cfull = 54/2162
        cquadra = 2/2162

elif pontos == 2:
    if num == 4: 
        cfull = 24024/103776
        cquadra = 552/103776
    elif num == 5:
        cfull = 358/2162
        cquadra = 4/2162

elif pontos == 3:
    if num == 3:
        cfull = 1628352/5085024
        cquadra = 415392/5085024
    elif num == 4:
        cquadra = 33924/103776
        cfull = 6492/103776
    elif num == 5:
        cfull = 630/2162
        cquadra = 92/2162

elif pontos == 6:
    cquadra = 94/2162

if pontos < 5:
    if num == 3:
        if mnaipe[1] == 3:
            cflush = (10*9*8)/(48*47*46)
        elif mnaipe[1] == 4:
            cflush = 1 - (38*37*36)/(48*47*46)
    elif num == 4:
        if mnaipe[1] == 3:
            cflush = (10*9*8)/(48*47*46)
        elif mnaipe[1] == 4:
            cflush = 1 - (38*37*36)/(48*47*46)
    elif num == 5:
        if mnaipe[1] == 3:
            cflush = (10*9)/(47*46)
        elif mnaipe[1] == 4:
            cflush = 1 - (38*37)/(47*46)
        
# chance de conseguir sequencia        
highmatch1 = [0,0]
if pontos < 4:
    sequencias1 = [['2','3','4','5','6'],['3','4','5','6','7'],['4','5','6','7','8'],
                  ['5','6','7','8','9'],['6','7','8','9','d'],['7','8','9','d','j'],
                  ['8','9','d','j','q'],['9','d','j','q','k'],['d','j','q','k','a']]
    for i in sequencias1:
        match1 = 0
        for n in i:
            rep = ['0', '0']
            repnum = 0
            for c in jogada:
                if c[0] == n and c[0] != rep[0] and c[0] != rep[1]:
                    match1 = match1 + 1
                    if repnum == 0:
                        repnum = 1
                        rep[0] = c[0]
                    else:
                        repnum = 0
                        rep[1] = c[0]
        if match1 >= highmatch1[0]:
            highmatch1[0] = match1
            highmatch1[1] = sequencias1.index(i)
    if num == 5:
        if highmatch1[0] == 4:
            if highmatch1[1] == 0 or highmatch1[1] == 8:
                cseq = 1 - (43*42)/(47*46)
            else:
                cseq = 1 - (39*38)/(47*46)
        elif highmatch1 == 3:
            if highmatch1[1] == 0 or highmatch1[1] == 8:
                cseq = (8*4)/(47*46)
            elif highmatch1[1] == 1 or highmatch1[1] == 7:
                cseq = 2*(8*4)/(47*46)
            else:
                cseq = 3*(8*4)/(47*46)
    if num == 6 and highmatch1[0] == 4:
        if highmatch1[1] == 0 or highmatch1[1] == 8:
            cseq = 4/46
        else:
            cseq = 8/46

# chance de conseguir straight flush
highmatch2 = [0,0]
if pontos < 8:
    for i in sequencias2:
        match2 = 0
        for n in i:
            for c in jogada:
                if c[0] == n and c[1] == mnaipe[0]:
                    match2 = match2 + 1
                        
        if match2 >= highmatch2[0]:
            highmatch2[0] = match2
            highmatch2[1] = sequencias2.index(i)
    if num == 5:
        if highmatch2[0] == 4:
            if highmatch2[1] == 0 or highmatch2[1] == 8:
                cstrflush = 1 - (46*45)/(47*46)
            else:
                cstrflush = 1 - (45*44)/(47*46)
        elif highmatch2 == 3:
            if highmatch2[1] == 0 or highmatch2[1] == 8:
                cstrflush = (2*1)/(47*46)
            elif highmatch2[1] == 1 or highmatch2[1] == 7:
                cstrflush = 2*(2*1)/(47*46)
            else:
                cstrflush = 3*(2*1)/(47*46)
    if num == 6 and highmatch2[0] == 4:
        if highmatch2[1] == 0 or highmatch2[1] == 8:
            cstrflush = 4/46
        else:
            cstrflush = 8/46        
            
if pontos < 1:     
    print('Par :', round(cpar*100, 2), '%')
if pontos < 2:
    print('Pares :', round(cpares*100, 2), '%')
if pontos < 3:
    print('Trinca :', round(ctrinca*100,2), '%')
if pontos < 4:
    print('Sequência :', round(cseq*100, 2), '%')
if pontos < 5:
    print('Flush :', round(cflush*100, 2), '%')
if pontos < 6:
    print('Full house :', round(cfull*100, 2), '%')
if pontos < 7:
    print('Quadra :', round(cquadra*100, 2), '%')
if pontos < 8:
    print('Straight flush :', round(cstrflush*100, 2), '%')

par de : k
Pares : 37.47 %
Trinca : 6.66 %
Sequência : 0 %
Flush : 34.97 %
Full house : 2.5 %
Quadra : 0.09 %
Straight flush : 0 %
//
//
todos têm um par de : k
Pares : 42.05 %
Trinca : 9.97 %
Sequência : 0 %
Flush : 0 %
Full house : 7.57 %
Quadra : 0.54 %
Straight flush : 0 %
